In [ ]:
import trino
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
conn = trino.dbapi.connect(
    host='trino', 
    port=8080,               
    user='TrinoPython',          
    catalog='iceberg',         
    schema='gold'           
)

cur = conn.cursor()
cur.execute("""
    SELECT id, symbol, date, close, label_1, label_2, label_3
    FROM fact_daily_ohlcv
    WHERE date >= DATE '2022-02-01'
""")

In [ ]:
df = pd.DataFrame(rows, columns=[col[0] for col in cur.description])
df

In [ ]:
df.info()

In [ ]:
def plot_movement_labels(df: pd.DataFrame, symbol: str, label_col: str):
    df_per_symbol = df[df['symbol'] == symbol].copy()
    
    plt.figure(figsize=(14,6))
    plt.plot(df_per_symbol['date'], df_per_symbol['close'], label='Close Price', color='blue')

    colors = {0:'red', 1:'orange', 2:'green'}
    for label in [0,1,2]:
        df_label = df_per_symbol[df_per_symbol[label_col] == label]
        plt.scatter(df_label['date'], df_label['close'], 
                    color=colors[label], label=f'label_{label}', s=40, alpha=0.4)

    plt.title(f"{symbol} Close Price with {label_col}")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.legend()
    plt.grid(True)
    plt.show()